In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import Image
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [35]:
generate_report = False
def plot(trace_num, x_data, y_data, xlable = 'xlable', ylable = 'ylable',
                legend = 'legend', title = 'title', mode='lines'):
    plot.counter += 1
    fig_name = 'images/' + str(plot.counter) + '.jpg'
    fig = go.Figure()
    for i in range(trace_num):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode=mode, name=legend[i], showlegend = True))
    fig.update_layout(
        title=go.layout.Title(
            text=title,
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlable
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylable
            )
        )
    )
    if generate_report is True:
        fig.write_image(fig_name)
        display(Image(fig_name))
    else:
        fig.show()
plot.counter = 0

In [36]:
def exp_smooth_forward(data, alpha, init=0):
    out = np.empty(data.size)
    out[0] = init
    for i in range(1, data.size):
        out[i] = out[i-1] + alpha*(data[i]-out[i-1])
    return out

In [37]:
def exp_smooth_backward(data, alpha, init=0):
    out = np.empty(data.size)
    out[-1] = init
    for i in reversed(range(data.size-1)):
        out[i] = out[i+1] + alpha*(data[i]-out[i+1])
    return out

In [38]:
def random_walk_tragectory(init, size, sigma):
    out = np.empty(size)
    out[0] = init
    w = np.random.normal(loc=0, scale=sigma, size=size)
    for i in range(1, size):
        out[i] = out[i-1]+w[i]
    return out

In [39]:
def measured_trajectory(data, sigma):
    return np.add(data, np.random.normal(loc=0, scale=sigma, size=data.size))

In [40]:
N = 300
sigma_w = 28
sigma_n = 97
num_points = np.linspace(1, N, num=N)
X = random_walk_tragectory(10, N, sigma_w)
z = measured_trajectory(X, sigma_n)

In [41]:
plot(2, [num_points, num_points], [X, z])

In [42]:
ksi = sigma_w**2/sigma_n**2
alpha = (-ksi + (ksi**2 + 4*ksi)**0.5)/2
print('Exponential smoothing parameter alpha = {:.2f}'.format(alpha))

Exponential smoothing parameter alpha = 0.25


In [43]:
exp_smooth_for = exp_smooth_forward(z, alpha, z[0])

In [45]:
exp_smooth_back = exp_smooth_backward(exp_smooth_for, alpha, exp_smooth_for[-1])

In [108]:
M = 7
running_mean = pd.Series(z).rolling(window = M, center = True).mean()
running_mean[:int(M/2)+1] = running_mean[int(M/2)]
running_mean[-int(M/2):] = running_mean[running_mean.size-int(M/2)-1]

In [109]:
plot(5, [num_points, num_points, num_points, num_points, num_points], 
        [X, z, exp_smooth_for, exp_smooth_back, running_mean],
     legend=['true trajectory', 'measurements', 'forward exponential smoothing',
             'backward exponential smoothing', 'running mean'])

In [110]:
def deviation(X, z):
    return np.sum((X[:]-z[:])**2)

def variability(X):
    I = np.empty(X.size-2)
    for i in range(X.size-2):
        I[i] = (X[i+2] - 2*X[i+1] + X[i])**2
    return np.sum(I)

In [117]:
dev_rm = deviation(running_mean, z)
dev_es = deviation(exp_smooth_back, z)
var_rm = variability(running_mean)
var_es = variability(exp_smooth_back)
print(('deviation indicator: running mean                     = {:.2f}\n'+
      '                     backward exponential smoothing   = {:.2f}').format(dev_rm, dev_es))
print(('variability indicator: running mean                   = {:.2f}\n'+
      '                       backward exponential smoothing = {:.2f}').format(var_rm, var_es))


deviation indicator: running mean                     = 2277444.72
                     backward exponential smoothing   = 2205593.62
variability indicator: running mean                   = 230344.35
                       backward exponential smoothing = 15301.92
